In [1]:
from core.board import Board
from analysis.msm_builder import get_msm_graph
from solver.solver import solve

In [2]:
# expert
r = 16
c = 30
n = 99

# r = 2
# c = 5
# n = 4
start = (0,0)
seed = None          
preset = False
profile = False
B_t = Board.random_complete(r,c,n,exclude=[start],seed=seed)
B_k = Board(r,c,n)
if preset:
    B_k.set_clear(0,0,1)
    B_k.set_clear(1,0,1)
    B_k.set_clear(1,3,3)
    B_k.set_mine(1,2)
    B_k.set_mine(1,4)
    pass
if preset: print(B_k)
elif seed is not None: print(B_t)

--------------------------------0/99
|11000002*3*2*21212*2*21101*2*1|
|*3100002*31223*2*21212*1022322|
|**31000122101*221101121112*23*|
|***200002*3132200001*1112*3*3*|
|*5*310002*3*2*100002221*444221|
|244*2000113242200012*123***100|
|*4**2111112*3*10012*321*4*4332|
|2**423*22*32*21112*3*111212***|
|3443*3*22*212212*4221100001232|
|**3*321111113*22**200000011100|
|224*30000001**213*21222222*100|
|003*3011100122101112**2**21100|
|014*522*210011100013*333310111|
|01****34*2002*20012*211*1001*1|
|123455*4*2124*2002*31022200222|
|1*11**3*211**21002*2001*1001*1|
--------------------------------


In [3]:
from analysis.msm_analysis import find_solutions, seperate_connected_msm_components
from analysis.probability import calculate_probabilities
from analysis.ordered_probability import calculate_safety
if preset:
    B_o, msm, c, f = get_msm_graph(B_k, order=1, verbose=0)
    #print(B_o, msm, c, f)
    msms = seperate_connected_msm_components(msm)
    #print(solutions := find_solutions(msms[0], verbose=0))
    #print(solutions.get_solution_counts_with_coords(clear_coords=[(0,1)]))
    #print(solutions.get_solution_counts_for_numbers((0,1), clear_coords=[(0,1)]))
    print(calculate_safety(B_k, order=5, prune_threshold=0.8, verbose=3))

In [4]:
import cProfile
if not preset and seed is not None:
    B_k = Board(r,c,n)
    if profile:
        cProfile.run('print(solve(B_k, B_t, first_move=start, verbose=0))')
    else:
        print(solve(B_k, B_t, first_move=start, verbose=2))

--------------------------------99/99
|1                             |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
|                              |
--------------------------------
Size of MSM Graph: 2
Size of MSM Graph: 2
tensor([[0.0000, 0.3333, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059,
         0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059,
         0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059,
         0.2059, 0.2059, 0.2059],
        [0.3333, 0.3333, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059, 0.2059,
 

In [5]:
import random
if seed is None and not preset:
    start = (0,0)
    max_iterations = 1000
    max_wins = 20
    wins = 0
    win_time = 0
    win_guesses = 0
    win_risk = 0
    ng_games = 0
    i = 0
    for i in range(max_iterations):
        seed = random.randint(0, 2**32)
        B_t = Board.random_complete(r,c,n,exclude=[start], seed=seed)
        B_k = Board(r,c,n)
        print(seed, end=' ')
        win, stats = solve(B_k, B_t, first_move=start, verbose=0)
        if win: 
            wins += 1
            win_time += stats.time
            win_guesses += stats.guesses
            win_risk += stats.risk
            if stats.guesses == 0: ng_games += 1
        print(win, f'{wins}/{i+1}', stats)
        if wins >= max_wins: break
    seed = None
    i += 1
    print(f'Wins: {wins/i}, Avg_Time: {win_time/wins}, Avg_guess: {win_guesses/wins}, Avg_risk:{win_risk/wins}, NG: {ng_games}')